# AAI614: Data Science & its Applications

*Notebook 3.2: Practice with Data Cleaning*

<a href="https://colab.research.google.com/github/harmanani/AAI614/blob/main/Week%203/Notebook3.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

Exercise I. Load the following datafile from GitHub

In [2]:
grads = pd.read_csv("https://raw.githubusercontent.com/harmanani/AAI614/main/Week%203/grads.csv")

In [3]:
grads

,Student Name,Avg Hours Studies per Week,GPA,University,Sense of Humour (0-5),Salary
0,George,20,NaN,NYU,3.0,$40k
1,Jerry,35,3.5,Columbia,5.0,$80k
2,Elaine,55,4.0,Columbia,4.2,$60k
3,Cosmo,5,2.0,City College,2.0,$25k
4,Newman,25,2.8,City College,0.0,$50k
5,Frank,35,3.0,Festivus Uni,NaN,$40k
6,Estelle,100,3.2,Festivus Uni,1.7,$0k
7,Leo,15,2.4,Festivus Uni,0.0,$35k
8,Rachel,50,4.0,Columbia,NaN,$75k


Question 1: Identify all the outliers in the above data.  Justify your answers using objective measures.

In [5]:
# 1. Cleaning Stage: Convert columns to numeric, turning errors (text) into NaN
numeric_cols = ['GPA', 'Salary', 'Avg Hours Studies per Week', 'Sense of Humour (0-5)']
for col in numeric_cols:
    # errors='coerce' turns strings like "$2,000" into NaN so they don't break the math
    grads[col] = pd.to_numeric(grads[col].astype(str).str.replace('[$,]', '', regex=True), errors='coerce')

# 2. IQR Function
def get_outliers(df, column):
    # Drop NaN values specifically for the calculation
    data = df[column].dropna()
    if data.empty: return pd.DataFrame()
    
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return df[(df[column] < lower) | (df[column] > upper)]

# 3. Execution
for col in numeric_cols:
    outliers = get_outliers(grads, col)
    print(f"--- Outliers in {col} ---")
    print(outliers if not outliers.empty else "No outliers found.")
    print("\n")

--- Outliers in GPA ---
No outliers found.


--- Outliers in Salary ---
No outliers found.


--- Outliers in Avg Hours Studies per Week ---
  Student Name  Avg Hours Studies per Week  GPA     University  \
6      Estelle                         100  3.2  Festivus Uni    

   Sense of Humour (0-5)  Salary  
6                    1.7     NaN  


--- Outliers in Sense of Humour (0-5) ---
No outliers found.




The IQR method is considered objective because it defines outliers based on the statistical distribution of the data itself, rather than relying on subjective assumptions

Question 2: There are various data that are missing.  Fill-in the missing data or delete the rows/columns that you think you should delete.  Justify your answer

In [7]:


# This removes rows with less than 3 fields filled
grads = grads.dropna(thresh=3)

# Fill the remaining gaps with the Median (Question 2 Strategy)
numeric_cols = ['GPA', 'Salary', 'Avg Hours Studies per Week', 'Sense of Humour (0-5)']

for col in numeric_cols:
    if grads[col].notna().any(): # Only calculate if there is at least one number
        grads[col] = grads[col].fillna(grads[col].median())
    else:
        grads[col] = grads[col].fillna(0) # Safety fill if column is empty

# Fill Categorical Data
grads['University'] = grads['University'].fillna(grads['University'].mode()[0])

print("Question 2 complete. Current null counts:")
print(grads.isnull().sum())

Question 2 complete. Current null counts:
Student Name                  0
Avg Hours Studies per Week    0
GPA                           0
University                    0
Sense of Humour (0-5)         0
Salary                        0
dtype: int64


I used dropna(thresh=3) to remove rows missing most of their values, since such rows lack enough information to support meaningful analysis in a small dataset.
Although the IQR analysis identified only one outlier, I chose median imputation instead of the mean because, in small datasets, even a single extreme value can skew the mean. The median better represents the typical student and keeps the data centered and unbiased.
I used the mode to fill missing values in the university column because it preserves the most common category in the dataset, maintains consistency, and avoids introducing artificial or unlikely categories.

Question 3: Reload the data and fill-in the data using mean method as well as the frequent method.

In [8]:
# Reloading the original list so that we have the Nan \
grads = pd.read_csv("https://raw.githubusercontent.com/harmanani/AAI614/main/Week%203/grads.csv")

# Clean numeric types
numeric_cols = ['GPA', 'Salary', 'Avg Hours Studies per Week', 'Sense of Humour (0-5)']
for col in numeric_cols:
    grads[col] = pd.to_numeric(grads[col].astype(str).str.replace('[$,]', '', regex=True), errors='coerce')

# Apply MEAN
for col in numeric_cols:
    grads[col] = grads[col].fillna(grads[col].mean())

# Apply the FREQUENT (Mode)
grads['University'] = grads['University'].fillna(grads['University'].mode()[0])

Exercise II. Run the cell below to create a new dataframe called `df_miss`.  Its first column will contain some missing values.

In [10]:
import pandas as pd
import numpy as np
import random

nrows = 10
ncols = 5

# set a seed for random number generation
np.random.seed(314)
# create an array filled with random data
data = np.array(np.random.rand(nrows, ncols))
# put the data to a pandas dataframe
df_miss = pd.DataFrame(data)
# rename the columns
df_miss.columns = ['col_'+str(ii) for ii in range(ncols)]

# randomly set some values to missing
ix0 = np.random.randint(nrows, size=3)
ix1 = np.random.randint(nrows, size=3)


df_miss.loc[ix0, 'col_0'] = np.nan
df_miss.loc[ix1, 'col_1'] = np.nan



#df_miss['col_0'][ix0] = np.nan
#df_miss['col_1'][ix1] = np.nan

print(df_miss)

      col_0     col_1     col_2     col_3     col_4
0       NaN       NaN  0.265048  0.783205  0.918001
1  0.827355       NaN  0.260480  0.911763  0.260757
2  0.766376  0.261531  0.122291  0.386006  0.840081
3       NaN       NaN  0.633110  0.584766  0.581232
4  0.677205  0.687155  0.438927  0.320927  0.570552
5       NaN  0.861074  0.834805  0.105766  0.060408
6  0.596882  0.792395  0.226356  0.535201  0.136066
7  0.372244  0.151977  0.429822  0.792706  0.406957
8  0.177850  0.909252  0.545331  0.100497  0.718721
9  0.978429  0.309776  0.260126  0.662900  0.139720


Impute the missing values (NaN) in `col_0` (but not `col_1`) with the median.  Store the values in the dataframe by using the parameter `inplace`.  Print the dataframe.

In [11]:
# median of the column
med_val = df_miss['col_0'].median()

#Fill the NaN values specifically in col_0 using the median, he parameter 'inplace=True' tells pandas to modify the existing df_miss object
df_miss['col_0'].fillna(value=med_val, inplace=True)

# Print the dataframe to verify the change
print("Dataframe after Task 1 (col_0 median):")
print(df_miss)

Dataframe after Task 1 (col_0 median):
      col_0     col_1     col_2     col_3     col_4
0  0.677205       NaN  0.265048  0.783205  0.918001
1  0.827355       NaN  0.260480  0.911763  0.260757
2  0.766376  0.261531  0.122291  0.386006  0.840081
3  0.677205       NaN  0.633110  0.584766  0.581232
4  0.677205  0.687155  0.438927  0.320927  0.570552
5  0.677205  0.861074  0.834805  0.105766  0.060408
6  0.596882  0.792395  0.226356  0.535201  0.136066
7  0.372244  0.151977  0.429822  0.792706  0.406957
8  0.177850  0.909252  0.545331  0.100497  0.718721
9  0.978429  0.309776  0.260126  0.662900  0.139720


C:\Users\layal\AppData\Local\Temp\ipykernel_32172\294856411.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_miss['col_0'].fillna(value=med_val, inplace=True)


Impute the missing values in `col_1` with value 0.  Store the values in the dataframe by using the parameter `inplace`.  Print the dataframe.

In [12]:
# Fill the NaN values in col_1 with a constant 0, using 'inplace=True' to update the dataframe directly
df_miss['col_1'].fillna(value=0, inplace=True)

# Print the final dataframe
print("Final Dataframe after Task 2 (col_1 zeros):")
print(df_miss)

Final Dataframe after Task 2 (col_1 zeros):
      col_0     col_1     col_2     col_3     col_4
0  0.677205  0.000000  0.265048  0.783205  0.918001
1  0.827355  0.000000  0.260480  0.911763  0.260757
2  0.766376  0.261531  0.122291  0.386006  0.840081
3  0.677205  0.000000  0.633110  0.584766  0.581232
4  0.677205  0.687155  0.438927  0.320927  0.570552
5  0.677205  0.861074  0.834805  0.105766  0.060408
6  0.596882  0.792395  0.226356  0.535201  0.136066
7  0.372244  0.151977  0.429822  0.792706  0.406957
8  0.177850  0.909252  0.545331  0.100497  0.718721
9  0.978429  0.309776  0.260126  0.662900  0.139720


C:\Users\layal\AppData\Local\Temp\ipykernel_32172\2070236010.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_miss['col_1'].fillna(value=0, inplace=True)
